In [1]:
#  Required imports

import math, os, sys, time

import numpy as np

from matplotlib import animation, pyplot as plt

from scipy import stats

from multiprocess import Process, Queue
from threading import Lock, Thread

import threading

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers    import Conv2D, Concatenate, Dense, Dropout, Flatten, Input, MaxPooling2D, Rescaling
from tensorflow.keras.models    import Model
from tensorflow.keras.callbacks import EarlyStopping

print("TensorFlow has found devices:")
for device in tf.config.list_physical_devices() :
    print(f"-  {device}")
    

/Users/Ste/miniforge3/envs/tf-sandbox-py3p9/lib/python3.9/site-packages/jax/_src/lib/__init__.py:33: UserWarning: JAX on Mac ARM machines is experimental and minimally tested. Please see https://github.com/google/jax/issues/5501 in the event of problems.
  warnings.warn("JAX on Mac ARM machines is experimental and minimally tested. "


TensorFlow has found devices:
-  PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
-  PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [ ]:
###
###  Global constants
###

#  Initially we will just run on a game board of fixed size, to avoid building an architecture to handle 
#  variable board sizes, so let's configure this here

horizontal_size = 30
vertical_size   = 20
horizontal_pad  = 5
vertical_pad    = 5
horizontal_max  = horizontal_size + 2*horizontal_pad
vertical_max    = vertical_size   + 2*vertical_pad

# reward = -1 per turn more interesting because it forces policy to learn optimal long-term return, not
#    just optimise the immediate return
reward_per_turn = -1
lambda_r        = 0
lambda_b        = -2.
gamma           = 1.

r_start = np.array([horizontal_pad, vertical_pad+vertical_size-1])
r_end   = np.array([horizontal_pad+horizontal_size-1, vertical_pad]) 
r_norm  = np.array([horizontal_max, vertical_max])

action_list = np.array([[-1, -1], [-1, 0], [-1, 1], [0, -1], [0, 0], [0, 1], [1, -1], [1, 0], [1, 1]])

print(f"r_start = {r_start}")
print(f"r_end   = {r_end}")


In [ ]:
###
###  Define and unit-test methods used to calculate Lp measures and norms
###


def Lp_norm(v, p=2) :
    '''
    Calculate Lp norm of vector v, defined as [sum_i v_i^p]^(1/p). If np.isfinite(p) returns False then
    calculate the L-infinity norm, which just returns the highest mod-vector-component.
    Inputs:
      > v, np.ndarray of any shape >= 1D
        vector to calculate the norm of
      > p, float, default=2
        exponent of the Lp norm
    Return:
      > float, the Lp norm
    '''
    ##  If type(v) is not numpy array then try to cast it to one
    if type(v) != np.ndarray :
        v = np.array(v)
    ##  Return L-infinity norm if p is not a real number
    if not np.isfinite(p) :
        return np.fabs(v).max()
    ##  Use np.power method to return Lp norm if p is finite
    return np.power(np.power(v, p).sum(), 1./p)
  
    
def Lp_distance(v1, v2, p=2) :
    '''
    Calculate Lp distance between vectors v1 and v2 by calling Lp_norm(v2-v1).
    Inputs:
      > v1, np.ndarray of any shape >= 1D
        first vector
      > v2, np.ndarray of same shape as v1
        second vector
      > p, float, default=2
        exponent of the Lp-distance
    Return:
      > float, the Lp distance
    '''
    ##  If v1 or v2 are not numpy arrays then try to cast them
    if type(v1) != np.ndarray :
        v1 = np.array(v1)
    if type(v2) != np.ndarray :
        v2 = np.array(v2)
    ##  Return the norm of the difference between v1 and v2
    return Lp_norm(v2 - v1, p=p)


V1 = np.array([2, 4, 5, 7])
V2 = np.array([5, 3, 6, 4])
dV = V2 - V1

print(f"UNIT TEST: Lp_norm({V1}, p=2) = {Lp_norm(V1, 2):.3f}  [expect {np.sqrt((V1**2).sum()):.3f}]")
print(f"UNIT TEST: Lp_norm({V1}, p=2.0) = {Lp_norm(V1, 2.0):.3f}  [expect {np.sqrt((V1**2).sum()):.3f}]")
print(f"UNIT TEST: Lp_norm({V1}, p=np.inf) = {Lp_norm(V1, np.inf):.3f}  [expect {np.fabs(V1).max():.3f}]")

print(f"UNIT TEST: Lp_distance({V1}, {V2}, p=2) = {Lp_distance(V1, V2, 2):.3f}  [expect {np.sqrt((dV**2).sum()):.3f}]")



In [ ]:
###
###  Define and unit-test environment methods
###


def perform_action(r_agent, action, base_reward=reward_per_turn, boundary_reward=lambda_b, 
                   dr_reward_factor=lambda_r, verbose=False, agent_is_normed=False) :
    '''
    Given the current environment and agent states, perform the specified action and return the reward 
    obtaine along with the new agent state.
    Inputs:
      > r_agent, np.ndarray of shape (2,)
        (x,y) position of agent at initial timestep
      > action, np.ndarray of shape (2,)
        (dx,dy) of action to be performed, each component expected to be one of {-1, 0, +1}
      > base_reward, float, default=-2.
        basic reward returned every turn (expected -ve)
      > boundary_reward, float, default=lambda_b
        reward received when encountering the edge of the game board (expected -ve)
      > dr_reward_factor, float, default=lambda_r
        factor multiplied by change-in-distance to calculate movement reward (expected +ve)
    Returns:
      > float
        reward obtained by performing action
      > np.ndarray of shape (2,)
        (x,y) position of agent at iterated timestep
    '''
    ##  Undo agent feature normalisation if needed
    if agent_is_normed :
        r_agent = r_agent.copy() * r_norm
    ##  Make sure initial state is valid to protect against unexpected behaviour
    if is_terminal(r_agent) :
        raise RuntimeError(f"Agent state is terminal, so no actions may be performed")
    if is_out_of_bounds(r_agent) :
        raise RuntimeError(f"Agent state {r_agent} is out of bounds, so no actions may be performed")
    ##  Get initial distance of agent from the end
    d_agent = Lp_distance(r_agent, r_end)
    ##  Iterate agent position
    ##  - if agent hits a wall then add an appropriate penalty and return agent to original position
    r_agent_p = r_agent + action
    reward_b  = 0
    if r_agent_p[0] < 0 or r_agent_p[0] >= horizontal_max or r_agent_p[1] < 0 or r_agent_p[1] >= vertical_max :
        reward_b  = boundary_reward
        r_agent_p = r_agent.copy()
    ##  Get distance-based reward
    d_agent_p = Lp_distance(r_agent_p, r_end)
    reward_r  = dr_reward_factor * (d_agent - d_agent_p) / np.sqrt(2)
    ##  Calculate total reward by summing the base, boundary and distance rewards
    reward = base_reward + reward_b + reward_r
    if verbose :
        print(f"perform_action: agent {r_agent} action {action} --> agent {r_agent_p} reward {reward:.2f}  [{base_reward:.2f} (base) + {reward_b:.2f} (b) + {reward_r:.2f} (r)]")
    ##  Return reward and new agent state
    return reward, r_agent_p


def is_terminal(r_agent) :
    '''
    Return True if the agent is in the terminal state and False otherwise.
    Inputs:
      > r_agent, np.ndarray of shape (2,)
        agent position as (x,y)-coordinates
    Returns:
      > bool
        whether the agent is in the terminal state
    '''
    if r_agent[0] == r_end[0] and r_agent[1] == r_end[1] :
        return True
    return False


def is_out_of_bounds(r_agent) :
    '''
    Return True if the agent isout of bounds and False otherwise.
    Inputs:
      > r_agent, np.ndarray of shape (2,)
        agent position as (x,y)-coordinates
    Returns:
      > bool
        whether the agent is out of bounds.
    '''
    if r_agent[0] <  0 : return True
    if r_agent[1] <  0 : return True
    if r_agent[0] >= horizontal_max : return True
    if r_agent[1] >= vertical_max    : return True
    return False
    

def get_greedy_action(r_agent, *q_models) :
    '''
    Sample a greedy action from the Q-value models provided. If multiple models provided then use their mean.
    Inputs:
      > r_agent, np.ndarray of shape (2,)
        agent position as (x,y)-coordinates
      > q_models, list of tf.keras Model class, each with inputs [agent position, action] = [Input(2), Input(2)]
        list of Keras Q(s,a) models
    Returns:
      > np.ndarray of shape (2,)
        action defined by greedy policy over the model(s) at this agent position
      > list of np.ndarray of shape (9,)
        action values in the same order as action_list, in list of models provided
    '''
    r_agents            = np.array([r_agent/r_norm for i in range(9)])
    model_args          = [r_agents, action_list]
    model_action_values = [model.predict(model_args) for model in q_models]
    action_values       = np.mean(model_action_values, axis=0)
    best_action         = action_list[np.argmax(action_values)]
    return best_action, model_action_values


def get_exploration_action(num=1) :
    '''
    Generate uniformly random actions from the 9 available.
    Input:
      > num, int, default=1
        number of random actions to generate
    Returns :
      > np.ndarray of size (num,2)
        list of actions generated
    '''
    return action_list[np.random.randint(low=0, high=8, size=(num,))]
    


r_agent, action = np.array([0, 5]), np.array([-1, 0])
print(f"UNIT TEST: perform_action({r_agent}, {action}, verbose=True) [expect boundary penalty]")
perform_action(r_agent, action, verbose=True)

r_agent, action = np.array([5, 0]), np.array([0, -1])
print(f"\nUNIT TEST: perform_action({r_agent}, {action}, verbose=True) [expect boundary penalty]")
perform_action(r_agent, action, verbose=True)

r_agent = r_end + np.array([-1,1])
print(f"\nUNIT TEST: perform_action({r_agent}, :, verbose=True) [expect cyclic r rewards]")
for action in action_list :
    perform_action(r_agent, action, verbose=True)

print(f"\nUNIT TEST: is_terminal({r_start}) --> {is_terminal(r_start)} [expect False]")

print(f"\nUNIT TEST: is_terminal({r_end}) --> {is_terminal(r_end)} [expect True]")



In [ ]:
###
###  Method for creating action-value model
###

def create_action_value_model(name=None) :
    '''
    Create a network for the action-value model.
    Inputs:
      > name, str, default=None
        model name, if None then keras default is used
    Returns:
      > keras Model: uncompiled keras model (must be trained using custom loop)
    '''
    input_layer_r = Input ((2,))
    input_layer_a = Input ((2,))
    next_layer_r  = Dense(25, activation="relu")(input_layer_r)
    next_layer_a  = Dense(25, activation="relu")(input_layer_a)
    next_layer    = Concatenate()([next_layer_r, next_layer_a])
    next_layer    = Dense(200, activation="relu")(next_layer)
    next_layer    = Dense(500, activation="relu")(next_layer)
    next_layer    = Dense(500, activation="relu")(next_layer)
    next_layer    = Dense(100, activation="relu")(next_layer)
    output_layer  = Dense(1, activation="linear")(next_layer)
    #output_layer  = Rescaling(-1.)(next_layer)
    model         = Model([input_layer_r, input_layer_a], output_layer, name=name)
    return model
    

In [ ]:
def generate_directory_for_file_path(fname, print_msg_on_dir_creation=True) :
    """
    Create the directory structure needed to place file fname. Call this before fig.savefig(fname, ...) to 
    make sure fname can be created without a FileNotFoundError
    Input:
       - fname: str
                name of file you want to create a tree of directories to enclose
                also create directory at this path if fname ends in '/'
       - print_msg_on_dir_creation: bool, default = True
                                    if True then print a message whenever a new directory is created
    """
    while "//" in fname :
        fname = fname.replace("//", "/")
    dir_tree = fname.split("/")
    dir_tree = ["/".join(dir_tree[:i]) for i in range(1,len(dir_tree))]
    dir_path = ""
    for dir_path in dir_tree :
        if len(dir_path) == 0 : continue
        if not os.path.exists(dir_path) :
            os.mkdir(dir_path)
            if print_msg_on_dir_creation :
                print(f"Directory {dir_path} created")
            continue
        if os.path.isdir(dir_path) : 
            continue
        raise RuntimeError(f"Cannot create directory {dir_path} because it already exists and is not a directory")
    
    
def create_greedy_policy_plot(*q_models, epoch_idx=-1, verbose=False, show=False, close=False, save="",
                              do_value_function=True) :
    '''
    Create a plt.Figure instance visualising the greedy policy defined by the average of the q-value models 
    provided. Allows for plot to be shown, saved and/or closed using plt interface. Returns the plot figure
    and axis objects so they can continue to be manipulated, but note that objects will no longer be in scope
    if we have called plt.close(fig).
    Inputs:
      > q_models, list of keras Model class
        list of q-value models to define the greedy policy
      > epoch_idx, int, default=-1
        if positive then draw a text box displaying how many epochs have been performed
      > verbose, bool, default=False
        if True then print some text to display progress as we evaluate the models for every state/action pair
      > show, bool, default=False
        if True then call plt.show(fig)
      > close, bool, default=False
        if True then call plt.close(fig)
      > save, str, default=""
        if string provided then call fig.savefig(save, ...), creating any required subdirectories if needed
    Returns:
      > plt.Figure instance
      > plt.Axes instance
    '''
     
    #  Keep track of how long plotting takes, to help inform how often to call this function    
    start_time = time.time()
    
    #  Set up plot
    fig = plt.figure(figsize=(12*horizontal_max/11.5,12*vertical_max/11.5))
    ax  = fig.add_subplot(1, 1, 1)
    ax.set_xlim(-0.5, horizontal_max-0.5)
    ax.set_ylim(-0.5, vertical_max-0.5)
    ax.tick_params(axis="both", which="both", right=True, top=True, direction="in", labelsize=16)
    
    #  Draw grid lines manually
    for x in range(horizontal_max-1) :
        ax.axvline(x+0.5, ls="-", c="gray")
    for y in range(vertical_max-1) :
        ax.axhline(y+0.5, ls="-", c="gray")
    
    #  Draw arrows by looping over states and finding greedy action according to q-models
    is_first_arrow = True
    for x in range(horizontal_max) :
        for y in range(vertical_max) :
            if x == r_end[0] and y == r_end[1] : continue
            r_agent  = np.array([x,y])
            dr_agent = Lp_distance(r_agent, r_end)
            if verbose :
                sys.stdout.write(f"\rEvaluating greedy policy for agent state ({x}, {y})".ljust(100))
            action, qs = get_greedy_action(r_agent, *q_models)
            q1s, q2s   = qs
            r_agent_p  = r_agent + action
            dr_agent_p = Lp_distance(r_agent_p, r_end)
            color      = "darkblue" if (dr_agent_p < dr_agent and not is_out_of_bounds(r_agent_p)) else "darkred"
            alpha      = 0.4 if do_value_function else 1
            dx, dy     = action
            if dx == 0 and dy == 0 :
                ax.plot(x, y, "o", markersize=8, c=color, alpha=alpha)
            else :
                ax.arrow(x - 0.3*dx, y - 0.3*dy, 0.6*dx, 0.6*dy, head_width=0.25, length_includes_head=True,
                         color=color, alpha=alpha)
                is_first_arrow = False
            if do_value_function :
                ax.text(x-0.47, y+0.47, f"{-q1s.min():.1f}", fontsize=10.5, ha="left" , va="top"   , weight="bold", alpha=0.7)  
                ax.text(x+0.47, y+0.47, f"{-q1s.max():.1f}", fontsize=10.5, ha="right", va="top"   , weight="bold", alpha=0.7)  
                ax.text(x-0.47, y-0.47, f"{-q2s.min():.1f}", fontsize=10.5, ha="left" , va="bottom", weight="bold", alpha=0.7)  
                ax.text(x+0.47, y-0.47, f"{-q2s.max():.1f}", fontsize=10.5, ha="right", va="bottom", weight="bold", alpha=0.7)       
                
    #  Draw accompanying plot objects
    ax.fill_between([r_start[0]-0.5, r_start[0]+0.5], r_start[1]-0.5, r_start[1]+0.5, color="g", alpha=0.2, label="Start")
    ax.fill_between([r_end  [0]-0.5, r_end  [0]+0.5], r_end  [1]-0.5, r_end  [1]+0.5, color="r", alpha=0.2, label="Finish")
    ax.legend(loc=(0.38,1.008), ncol=3, fontsize=16, frameon=False)
    
    #  Draw text boxes displaying title and num. epochs
    ax.text(0, 1.01, "Min/max $q(s,a)$ and greedy policy per $s$", transform=ax.transAxes, 
            fontsize=18, weight="bold", ha="left", va="bottom")
    if epoch_idx >= 0 :
        ax.text(1, 1.01, f"After {epoch_idx} epochs", ha="right", va="bottom", weight="bold", 
                transform=ax.transAxes, fontsize=16)
    
    #  Verbose messaging
    if verbose :
        sys.stdout.write(f"\nPlot created in {time.time()-start_time:.2f}s".ljust(100)+"\n")
       
    #  Save / show / close
    if len(save) > 0 :
        generate_directory_for_file_path(save)
        plt.savefig(save, bbox_inches="tight")
    if show :
        plt.show(fig)
    if close :
        plt.close(fig)
        
    #  Return figure and axis
    return fig, ax


def create_training_curves_plot(loss_record, ref_loss_record, maxQ_record, show=False, close=False, save="") :
    '''
    Create a plt.Figure instance visualising the training curves. Allows for plot to be shown, saved and/or 
    closed using plt interface. Returns the plot figure and axis objects so they can continue to be 
    manipulated, but note that objects will no longer be in scope if we have called plt.close(fig).
    Inputs:
      > q_models, list of keras Model class
        list of q-value models to define the greedy policy
      > verbose, bool, default=False
        if True then print some text to display progress as we evaluate the models for every state/action pair
      > show, bool, default=False
        if True then call plt.show(fig)
      > close, bool, default=False
        if True then call plt.close(fig)
      > save, str, default=""
        if string provided then call fig.savefig(save, ...), creating any required subdirectories if needed
    Returns:
      > plt.Figure instance
      > plt.Axes instance (axis corresponding to loss curves)
      > plt.Axes instance (axis corresponding to ref_loss curves)
      > plt.Axes instance (axis corresponding to maxQ curves)
    '''
    
    def draw_curve(ax, container, m, c, label) :
        ax.plot([x for x,y in container], [y for x,y in container], m, ms=7, c=c, alpha=1.0, label=label)
        for ((x1,y1), (x2,y2)) in zip(container[:-1], container[1:]) :
            if np.fabs(x2-x1) > 1.5 : continue
            ax.plot([x1, x2], [y1, y2], "-", c=c, lw=2, alpha=0.7)
            
    fig = plt.figure(figsize=(30,15))
    
    ax1 = fig.add_subplot(3, 1, 1)
    ax1.tick_params(axis="both", which="both", right=True, top=True, direction="in", labelsize=30)
    ax1.set_title(r"Mean loss per batch [$(1-\lambda)\cdot$batch + $\lambda\cdot$ref]", fontsize=30)
    ax1.xaxis.set_ticklabels([])
    draw_curve(ax1, loss_record["Q1"], "o", "r", "$q_1$")
    draw_curve(ax1, loss_record["Q2"], "x", "b", "$q_2$")
    ax1.set_yscale("log")
    ax1.legend(loc=(0,1.02), fontsize=30, ncol=3, title_fontsize=30)
    ax1.grid()
    
    ax2 = fig.add_subplot(3, 1, 2)
    ax2.tick_params(axis="both", which="both", right=True, top=True, direction="in", labelsize=30)
    ax2.set_title(r"Mean loss per batch [ref only]", fontsize=30)
    ax2.xaxis.set_ticklabels([])
    draw_curve(ax2, ref_loss_record["Q1"], "o", "r", "$q_1$")
    draw_curve(ax2, ref_loss_record["Q2"], "x", "b", "$q_2$")
    ax2.set_yscale("log")
    ax2.grid()
    
    ax3 = fig.add_subplot(3, 1, 3)
    ax3.tick_params(axis="both", which="both", right=True, top=True, direction="in", labelsize=30)
    ax3.set_title(r"Max $|q(s,a)|$ over all batches", fontsize=30)
    ax3.set_xlabel(r"Epoch index", labelpad=15, fontsize=30)
    draw_curve(ax3, maxQ_record["Q1"], "o", "r", "$q_1$")
    draw_curve(ax3, maxQ_record["Q2"], "x", "b", "$q_2$")
    ax3.axhline(0, ls="--", lw=2, c="gray")
    true_max = max([horizontal_pad+horizontal_size, vertical_pad+vertical_size]) + np.fabs(lambda_b)
    ax3.axhline(true_max, ls="--", lw=2, c="gray")
    ax3.text(0, 0.99*true_max, "True maximum", ha="left", va="top", fontsize=30, c="gray")
    ax3.grid()
    
    fig.subplots_adjust(hspace=0.2)
    
    if len(save) > 0 :
        generate_directory_for_file_path(save)
        plt.savefig(save, bbox_inches="tight")
    if show :
        plt.show(fig)
    if close :
        plt.close(fig)
        
    return fig, ax1, ax2, ax3
    

In [ ]:

def test_models(r_agent, *q_models) :
    r_agents       = np.array([r_agent/r_norm for i in range(9)])
    model_args     = [r_agents, action_list]
    action_values  = np.mean([model.predict(model_args) for model in q_models], axis=0).flatten()
    str_actions    = '  '.join([f"{a}".ljust(8) for a in action_list])
    str_values     = '  '.join([f"{q:.3f}".ljust(8) for q in action_values])
    print(f"TEST: values at {r_agent}")
    print(f"TEST: {str_actions}")
    print(f"TEST: {str_values}")
    

In [ ]:
###
###  Identify priority state/action pairs
###

priority_states, priority_actions, priority_returns = [], [], []
for action in action_list :
    r_initial = r_end - action
    if is_terminal(r_initial) : continue
    reward, _ = perform_action(r_initial, action)
    priority_states .append(r_initial / r_norm)
    priority_actions.append(action)
    priority_returns.append(reward)
    
priority_states  = np.array(priority_states )
priority_actions = np.array(priority_actions)
priority_returns = np.array(priority_returns)

print(f"Found {len(priority_returns)} priority state-action pairs with returns:  {'  '.join([f'{x:.2f}' for x in priority_returns])}")

def print_priority_values(q1_model, q2_model) :
    model_args    = [priority_states, priority_actions]
    q1_values     = q1_model.predict(model_args).flatten()
    q2_values     = q2_model.predict(model_args).flatten()
    str_returns   = '  '.join([f"{q:.3f}".ljust(8) for q in priority_returns])
    str_q1_values = '  '.join([f"{q:.3f}".ljust(8) for q in q1_values])
    str_q2_values = '  '.join([f"{q:.3f}".ljust(8) for q in q2_values])
    print(f"EXP: {str_returns}")
    print(f"Q1 : {str_q1_values}")
    print(f"Q2 : {str_q2_values}")


In [ ]:
def create_config(config_fname, q1_model, q2_model, to_stdout=True) :
    '''
    Print environment, training and model configurations to file config_fname. Also print environment and
    training configurations to sys.stdout if requested, but do not print model summaries as they are verbose.
    Inputs:
      > config_fname, str
        name of config file to create
      > q1_model, keras Model
        first q-value model
      > q2_model, keras Model
        second q-value model
      > to_stdout, bool, default=True
        if True then repeat environment and training configurations to sys.stdout
    Returns:
      > None
    '''
    # Create message as list of strings
    config_message = []
    config_message.append(f"="*114 + "\n")
    config_message.append(f"Environment config:\n")
    config_message.append(f"> horizontal_size: {horizontal_size}\n")
    config_message.append(f"> vertical_size: {vertical_size}\n")
    config_message.append(f"> horizontal_pad: {horizontal_pad}\n")
    config_message.append(f"> vertical_pad: {vertical_pad}\n")
    config_message.append(f"> horizontal_max: {horizontal_max}\n")
    config_message.append(f"> vertical_max: {vertical_max}\n")
    config_message.append(f"> reward_per_turn: {reward_per_turn:.6f}\n")
    config_message.append(f"> lambda_r: {lambda_r:.6f}\n")
    config_message.append(f"> lambda_b: {lambda_b:.6f}\n")
    config_message.append(f"> gamma: {gamma:.6f}\n")
    config_message.append(f"="*114 + "\n")
    config_message.append(f"Training config:\n")
    config_message.append(f"> Using bootstrap method: {bootstrap_method}\n")
    config_message.append(f"> Using epochs of length {num_states}\n")
    config_message.append(f"> Updating gradient every batch of size {batch_size}\n")
    config_message.append(f"> Using optimizer_q1 {optimizer_q1} with learning rate {learning_rate:.6}\n")
    config_message.append(f"> Using optimizer_q2 {optimizer_q2} with learning rate {learning_rate:.6}\n")
    config_message.append(f"> Testing every {test_after_epochs} epochs\n")
    config_message.append(f"> Plotting policy every {plot_policy_after_epochs} epochs\n")
    config_message.append(f"> Plotting monitors every {plot_monitors_after_epochs} epochs\n")
    config_message.append(f"> Swapping q1 and q2 every {switch_after_epochs} epochs\n")
    config_message.append(f"> Cloning q2 from q1 every {clone_after_epochs} epochs\n")
    config_message.append(f"> Assigning a weight of {priority_weight} to anchoring state/action pairs\n")
    config_message.append(f"="*114 + "\n")
    # Make sure directory exists for file
    generate_directory_for_file_path(config_fname, print_msg_on_dir_creation=True)
    # Open file and print messages, also to stdout if configured
    # - also print q-model summaries, only to file
    with open(config_fname, "w") as config_file :
        for line in config_message :
            config_file.write(line)
            if not to_stdout : continue
            sys.stdout.write(line)
        config_file.write("\nModel configs:\n\n")
        q1_model.summary(print_fn=lambda x: config_file.write(x + '\n'))
        config_file.write("\n")
        q2_model.summary(print_fn=lambda x: config_file.write(x + '\n'))


- Use two optimisers so momentum from q1 does not propagate onto q2 or vice versa. Downside is that momentum skips into the future. Also we do nothing to solve the instability which comes from a constantly shifting objective
- Prefer to study rewards of "-1 per turn" instead of "reward for moving towards goal" because the first one is more generalisable and forces us to find techniques which optimise long-term return, whereas the second objective can be achieved by simply optimising immediate rewards and so it is not clear to what degree we are driven by long-term rather than short-term returns
- currently learning without momentum (SGD) as I'm worried about the impact that momentum might have on encouraging divergence (could be studied)
- What is happening when instantaneous divergences occur? Seems like things are going well then suddenly both batch loss and reference loss both explode
- Even though we know all returns must be negative definite, we still use a linear output layer for the network so the initial returns are symmetric about 0, so initial bootstraps do not start propagating chains of negative numbers which may encourage divergence (could be studied)
- Greedy policy exists in regions of aligned policies, reflecting smoothness of value function in (NxMx9) space
- Not using validation split because don't want there to be state/action pairs which agent does not have access to during training
- To what degree do we want each model to converge on the train data + current bootstrap target before we swap, thus updating the bootstrap target for the next model (and so on)
- Idea to improve convergence?
- Could get stable solution then study variations such as
    - changing optimizer (momentum and learning rate)
    - changing output layer (linear vs -1*relu)
    - reduce number of batch updates before switching, so each model does not converge before being used as bootstrap target
    - priority weight high vs low
    - -1 per turn vs dr per turn
    - small NN model
    - return clipping to reduce divergence
    - with/without boundary penalty
- Could derive optimal value function and use as absolute measure of quality
- Idea: form a validation set of state/action pairs and track their q-values to study divergence. E.g. all 9 actions at five different states (2 on boundary, 1 near middle)
- Idea: monitoring plot: mean q(s,a) across a for every q (maybe as heatmap, see how value function is evolving across state space. Suspect divergence driven by uniform raising of expected values leading to runaway process, not a slow development of a cone with a peak-value around r_end and more negative values concentrically around it.
- Use true DQN/double-Q method and freeze the same network for the bootstraps?
- Idea: Could make simple 1D function update example to show where divergence comes from, and study how to control it. E.g. line walk with boundary (same as current example but in 1D, actions are L or R)
- Important note: previous experiments impacted by bug in which y was profiled up to horizontal_max instead of vertical_max, ruining everything
- I think using two networks increases variance but is less biased
- Some kind of adaptive learning_rate would be nice
- Expect high variance to trigger divergence because we increase the distance between the function estimates and the truth
- Whether using two networks or a frozen network, having two players inherently causes instability because of the constantly shifting objective function, making it difficult to converge because each time we switch bootstrap function we change the objective (could be mitigated with slower learning?)
- We do not prioritise accuracy of states near to finish, and these get pulled around by nearby datapoints due to function approximation. These instabilities then propagate through the state space to "earlier" states, because we are using bootstrapping. If these final states never converge, then the function may constantly fluctuate-and-propagate forever without converging towards a stable solution. This is not necessarily a variance issue, rather that the function approximation allows all states to pull on all other ones, although variance may also contribute. Maybe solution is to assign high weight to priority states? This reduces learning rate elsewhere but may more effectively anchor the function in these important states, so the information which propagates back to other states (through bootstrapping) is good information. It does not fully solve the problem though, because functional approximations still lead to propagating mis-information in the rest of the state space. I expect that the further you get from the terminal state, the worse this problem becomes, because the amount of misinformation compounds over distance. When we compare the action-value-estimates at each epoch with the max-Q at that stage in training, we see that the final few states are fluctuating a lot. I don't see how this problem can be avoided without curating a training schedule to focus on and freeze states in a custom manner, which requires even more expert knowledge (on top of the priority state definitions already provided) when defeats the whole purpose.
- Why does off-policy behaviour contribute to divergence? I guess we are biasing the distribution of visited state-action pairs to give too much weight to actions which are not important, forcing the q-model to focus on these. Maybe if we behaved on-policy then we could find a steady-state solution which, although not necessarily exact (still using function approximation), is at least convergent?
- Bootstrapping does not do anything to correct the functional dependence which is already there. E.g. if the true q-function has a positive gradient, and the estimate has a negative gradient, then bootstrapping will continue to provide q-models with a negative gradient - unless we use a tailored reward signal (which has the two negative effects (i) introduces expert knowledge and (ii) biases the learned policy towards those the expert already knows). If we use a reward signal of e.g. '-1 per turn', the only learning pressure towards correcting the functional form comes from the states which do not bootstrap and so must be fixed in scale to their immediate reward, i.e. those next to terminal states. However for large state spaces, without priority weighting, these state-action pairs only carry a small weight and so provide a very small learning pressure. This means that the larger pressure - to reinforce 

In [ ]:

## Configure

num_epochs                 = -1
batch_size                 = 50
learning_rate              = 1e-3
plot_policy_after_epochs   = 50
plot_monitors_after_epochs = 50
switch_after_epochs        = -1
clone_after_epochs         = 1
test_after_epochs          = 50
priority_weight            = 0.3
bootstrap_method           = "clone"       # ["clone", "self", "other"]

%matplotlib inline


## Set up

loss_fcn     = tf.keras.losses.MeanSquaredError()
optimizer_q1 = tf.keras.optimizers.Adam(learning_rate=learning_rate)
optimizer_q2 = tf.keras.optimizers.Adam(learning_rate=learning_rate)
q1_model     = create_action_value_model(name="action_value_model_1")
q2_model     = q1_model if bootstrap_method == "self" else create_action_value_model(name="action_value_model_2")
states       = []
for x in range(horizontal_max) :  # range(r_end[0]-2, r_end[0]+3) :       # range(horizontal_max) : 
    for y in range(vertical_max) :  # range(r_end[1]-2, r_end[1]+3) :   # range(horizontal_max) : 
        r_agent = np.array([x,y])
        if is_terminal(r_agent) : continue
        states.append( r_agent / r_norm )
num_states = len(states)
if bootstrap_method not in ["clone", "self", "other"] :
    raise NotImplementedError(f"Bootstrap method {bootstrap_method} not implemented")


## Print config to file and screen (model summaries only to file because they are verbose)
create_config("figures/Helicopter_NB1/config.txt", q1_model, q2_model, to_stdout=True)


## Start training

start_time = time.time()
model_key_q1, model_key_q2 = "Q1", "Q2"  # used to keep track of which model is being traing each epoch
loss_record, ref_loss_record, maxQ_record = {"Q1":[], "Q2":[]}, {"Q1":[], "Q2":[]}, {"Q1":[], "Q2":[]}

epoch_idx = 0
while epoch_idx < num_epochs or num_epochs < 0 :
    
    # Determine whether to print test information
    if test_after_epochs > 0 and epoch_idx % test_after_epochs == 0 :
        #test_models(r_start, q1_model, q2_model)
        #test_models(r_end + np.array([-1,1]), q1_model, q2_model)
        print_priority_values(q1_model, q2_model)
    
    # Determine whether to plot current greedy policy
    if plot_policy_after_epochs > 0 and epoch_idx % plot_policy_after_epochs == 0 :
        create_greedy_policy_plot(q1_model, q2_model, epoch_idx=epoch_idx, verbose=True, show=True, close=True,
                                  do_value_function=True,
                                  save=f"figures/Helicopter_NB1/greedy_policy_epoch{epoch_idx}.pdf")
    
    # Determine whether to plot training curves
    if plot_monitors_after_epochs > 0 and epoch_idx > 0 and epoch_idx % plot_monitors_after_epochs == 0 :
        create_training_curves_plot(loss_record, ref_loss_record, maxQ_record, show=True, close=True,
                                    save="figures/Helicopter_NB1/training_curves.pdf")
        
    # Determine whether to switch q1 and q2
    # - keeping this irregular reduces the risk of feedback function<->bootstrap loops which drive divergence
    if bootstrap_method == "other" and switch_after_epochs > 0 and epoch_idx > 0 and epoch_idx % switch_after_epochs == 0 :
        model_key_q1, model_key_q2 = model_key_q2, model_key_q1
        q1_model, q2_model         = q2_model, q1_model
        optimizer_q1, optimizer_q2 = optimizer_q2, optimizer_q1
        
    # Determine whether to copy q1 to q2
    if bootstrap_method == "clone" and clone_after_epochs > 0 and epoch_idx % clone_after_epochs == 0 :
        q2_model.set_weights(q1_model.get_weights()) 
    
    sys.stdout.write(f"Epoch {epoch_idx+1} / {num_epochs}  [t={time.time()-start_time:.2f}s]")
    
    # Shuffle states and loop over batches, performing one gradient update per batch
    # - this has lower variance than applying one gradient update per sample, and also allows parallelisation
    np.random.shuffle(states)
    epoch_losses, ref_losses, max_abs_q_values = [], [], []
    for batch_idx in range(math.ceil(num_states/batch_size)) :
        # Resolve sample indices to be used for this batch update
        batch_idx_low, batch_idx_high = batch_idx*batch_size, min((batch_idx+1)*batch_size, num_states)
        actual_batch_size = batch_idx_high - batch_idx_low
        if actual_batch_size == 0 : continue
        
        # Update the current epoch message to keep track of batch progress 
        sys.stdout.write(f"\rEpoch {epoch_idx+1} / {num_epochs} batch indices ({batch_idx_low}, {batch_idx_high}) / {num_states}  [t={time.time()-start_time:.2f}s]")
        
        # Get batch of states and generate a random exploration action for each
        r_agents = np.array(states[batch_idx_low:batch_idx_high])
        actions  = get_exploration_action(num=actual_batch_size)
        
        # For each state/action pair, apply the action to get the immediate reward, and also find the
        # greedy action in the next state from which to calculate bootstraps
        rewards, r_agents_p, actions_p, agent_p_is_terminal = [], [], [], []
        for r_agent, action in zip(r_agents, actions) :
            reward, r_agent_p = perform_action(r_agent, action, agent_is_normed=True)
            agent_p_is_terminal.append(True if is_terminal(r_agent_p) else False)
            action_p, _ = get_greedy_action(r_agent_p, q1_model)
            r_agent_p /= r_norm
            rewards   .append(reward)
            r_agents_p.append(r_agent_p)
            actions_p .append(action_p)
        rewards, r_agents_p, actions_p = np.array(rewards), np.array(r_agents_p), np.array(actions_p)
        
        # Calculate obs_returns using the observed immediate rewards plut gamma * bootstrap values
        bootstrap_values = q2_model.predict([r_agents_p, actions_p]).flatten()
        bootstrap_values = np.array([0. if is_term else q for is_term,q in zip(agent_p_is_terminal,bootstrap_values)])
        obs_returns      = rewards + gamma * bootstrap_values
        
        # Calculate weights to apply to regular batch and priority samples
        sample_weights   = (1.-priority_weight) * np.full(shape=(len(r_agents),), fill_value=1./len(r_agents))
        priority_weights = priority_weight * np.full(shape=(len(priority_states),), fill_value=1./len(priority_states))
        
        # Concatenate regular batch and priority samples
        if priority_weight > 0 and priority_weight <= 1 :
            sample_weights   = np.concatenate([sample_weights , priority_weights]).flatten()
            r_agents         = np.concatenate([r_agents       , priority_states ])
            actions          = np.concatenate([actions        , priority_actions])
            obs_returns      = np.concatenate([obs_returns    , priority_returns]).flatten()
        
        # When using sample weights, we have to be careful with the object shapes. The loss function will
        # expect y_pred of shape (N,1) and y_true of shape (N,1), when the output shape is (1,), and sample
        # weights of shape (N,). When not using sample weights we can get away with being lazy on the shapes
        # of y_pred and y_true, but if we do this here then it will not correctly apply the correct sample
        # weight to the correct sample. Furthermore, MSE with sample weights will calculate mean(sw*res**2)
        # rather than normalising according to sum(sw*res**2)/sum(sw). This means we must also multiply sw 
        # by a factor of len(sw)/sum(sw) if we are to recover the MSE value that we expect. Note that we have
        # sum(sw)=1 by our construction above, but I still write it explicitly so the general idea is clear.
        sample_weights = sample_weights * len(sample_weights) / sample_weights.sum()
        obs_returns    = obs_returns.reshape((len(obs_returns),1))
        
        # ref_loss is the loss over only the priority state/action pairs, before gradient updates for
        # consistency with regular loss record. Since regular loss mixes (i) how close the priority samples
        # are to their correct values and (ii) how close other points are to their bootstrap-biased values,
        # the ref_loss removes the bootstrap samples so we can see whether these points are diverging
        ref_returns = q1_model([priority_states, priority_actions], training=False)
        ref_loss    = loss_fcn(priority_returns.reshape((len(ref_returns),1)), ref_returns.numpy())
        ref_losses.append(ref_loss)
                                
        # Apply gradient updates
        with tf.GradientTape() as tape:
            pred_returns     = q1_model([r_agents, actions], training=True)
            loss_value       = loss_fcn(obs_returns, pred_returns, sample_weight=sample_weights)
            grads            = tape.gradient(loss_value, q1_model.trainable_weights)
            optimizer_q1.apply_gradients(zip(grads, q1_model.trainable_weights))
            epoch_losses.append(loss_value.numpy())
            np_returns = np.fabs(pred_returns.numpy().flatten())
            max_q_idx  = np.argmax(np_returns)
            #sys.stdout.write(f"\nMax batch return at index {max_q_idx} state {r_agents[max_q_idx]*r_norm} action {actions[max_q_idx]} with q-value {np_returns[max_q_idx]:.1f}\n")
            max_abs_q_values.append(np_returns.max())
                                
    # Print epoch summary and end stdout line to keep this on screen
    epoch_mean_loss, epoch_mean_ref_loss, epoch_maxQ = np.mean(epoch_losses), np.mean(ref_losses), np.max(max_abs_q_values)
    sys.stdout.write(f"\rEpoch {epoch_idx+1} / {num_epochs}  [t={time.time()-start_time:.2f}s]  <loss = {epoch_mean_loss:.5f}, ref_loss = {epoch_mean_ref_loss:.5f}, max_Q = {epoch_maxQ:.1f}>".ljust(100)+"\n")
    loss_record    [model_key_q1].append((epoch_idx, epoch_mean_loss    ))
    ref_loss_record[model_key_q1].append((epoch_idx, epoch_mean_ref_loss))
    maxQ_record    [model_key_q1].append((epoch_idx, epoch_maxQ         ))
                              
    # Manually iterate epoch index and make sure stdout not lagging
    epoch_idx += 1
    sys.stdout.flush()
        